## Transfer Learning

In [5]:
from tensorflow import keras
import tensorflow as tf

fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [13]:
model_A = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")], name="Mnist_Model")

model_A.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
model_A.summary()

Model: "Mnist_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_16 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_17 (Dense)             (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [14]:
history = model_A.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid))

Epoch 1/100
1719/1719 [==============================] - 3s 2ms/step - loss: 0.6976 - accuracy: 0.7692 - val_loss: 0.5008 - val_accuracy: 0.8300
Epoch 2/100
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4864 - accuracy: 0.8299 - val_loss: 0.4466 - val_accuracy: 0.8458
Epoch 3/100
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4415 - accuracy: 0.8462 - val_loss: 0.4239 - val_accuracy: 0.8542
Epoch 4/100
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4135 - accuracy: 0.8539 - val_loss: 0.4121 - val_accuracy: 0.8528
Epoch 5/100
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3948 - accuracy: 0.8611 - val_loss: 0.3861 - val_accuracy: 0.8692
Epoch 6/100
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3784 - accuracy: 0.8658 - val_loss: 0.3806 - val_accuracy: 0.8668
Epoch 7/100
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3653 - accuracy: 0.8709 - val_loss: 0.3720 - val_ac

In [24]:
# model_A = keras.models.load_model("model_path.h5") # if we had to load another model
# we have to clone model_A , so when we train model_B it will not change also the layers of model_A
model_A_clone = keras.models.clone_model(model_A) #clone just the archi not the weights
model_A_clone.set_weights(model_A.get_weights())
model_A_clone.summary()

Model: "Mnist_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_16 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_17 (Dense)             (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [27]:
model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation='sigmoid'))
model_B_on_A.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_16 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 101       
Total params: 265,701
Trainable params: 265,701
Non-trainable params: 0
_________________________________________________________________


In [29]:
# freez the reused layers
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False
model_B_on_A.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
model_B_on_A.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_16 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 101       
Total params: 265,701
Trainable params: 101
Non-trainable params: 265,600
_________________________________________________________________


Train the model for few epochs with freezed layers: this will train the new layers 
we chould use train and validation sets of the new task B,
but here is just an exmple so we use the same data as model A

In [30]:
model_B_on_A.fit(X_train, y_train, epochs=4, validation_data=(X_valid, y_valid))
# NOW we can unfreez the resued layers and train the whole NN on the task B
for layer in model_B_on_A.layers:
    layer.trainable = True

Epoch 1/4


InvalidArgumentError:  Received a label value of 9 which is outside the valid range of [0, 1).  Label values: 3 6 1 8 5 8 3 7 1 1 1 7 9 0 4 2 7 7 5 3 1 9 7 0 4 2 0 1 9 6 8 7
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-30-0728a3e32ba4>:4) ]] [Op:__inference_train_function_449783]

Function call stack:
train_function


In [ ]:
# good practice to use small learning rate to avoid damaging the weights learned from task A    
optimizer = keras.optimizers.SGD(lr=1e-4)
model_B_on_A.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model_B_on_A.fit(X_train, y_train, epochs=16, validation_data=(X_valid, y_valid))

Transfert learning is better when used for convolutional neural nets; 
because these netx tend to learn general feature detectors in lower layers

# Unsupervised Pretraining